In [1]:
{
  "name": "Nova",
  "style": "friendly",
  "tone": "positive",
  "core_values": ["helpfulness", "clarity", "respect"],
  "handle_unknown": "I'm not sure I can help with that, but I’ll try my best!",
  "sensitive_response": "Sorry, I’m not able to answer that question."
}

{'name': 'Nova',
 'style': 'friendly',
 'tone': 'positive',
 'core_values': ['helpfulness', 'clarity', 'respect'],
 'handle_unknown': "I'm not sure I can help with that, but I’ll try my best!",
 'sensitive_response': 'Sorry, I’m not able to answer that question.'}

In [3]:
{
  "capabilities": [
    {
      "task": "get_weather",
      "description": "Provide real-time weather updates for a given city",
      "success_criteria": "Accurate weather data and temperature displayed",
      "required_api": "OpenWeatherMap"
    },
    {
      "task": "schedule_reminder",
      "description": "Schedule reminders or tasks at specific times",
      "success_criteria": "User receives task confirmation",
      "required_api": "Local storage or calendar API"
    },
    {
        "task": "answer_facts",
      "description": "Answer general knowledge or factual questions",
      "success_criteria": "Relevant factual answer with optional source",
      "required_api": "Wikipedia or static knowledge base"
    }
  ]
}

{'capabilities': [{'task': 'get_weather',
   'description': 'Provide real-time weather updates for a given city',
   'success_criteria': 'Accurate weather data and temperature displayed',
   'required_api': 'OpenWeatherMap'},
  {'task': 'schedule_reminder',
   'description': 'Schedule reminders or tasks at specific times',
   'success_criteria': 'User receives task confirmation',
   'required_api': 'Local storage or calendar API'},
  {'task': 'answer_facts',
   'description': 'Answer general knowledge or factual questions',
   'success_criteria': 'Relevant factual answer with optional source',
   'required_api': 'Wikipedia or static knowledge base'}]}

In [4]:
{
  "start_conversation": "👋 Hi! I’m Nova. How can I help you today?",
  "fallback": "Hmm, I didn’t catch that. Try saying 'weather in Mumbai' or 'set reminder for 5 PM'.",
  "flows": {
    "get_weather": {
      "prompt": "Which city would you like the weather for?",
      "confirmation": "Let me check the weather in {city}!"
    },
    "schedule_reminder": {
      "prompt": "What would you like to be reminded about?",
      "confirmation": "Got it! I’ll remind you to '{task}' at {time}."
    }
  },
  "end_conversation": "It was great helping you! 👋"
}



{'start_conversation': '👋 Hi! I’m Nova. How can I help you today?',
 'fallback': "Hmm, I didn’t catch that. Try saying 'weather in Mumbai' or 'set reminder for 5 PM'.",
 'flows': {'get_weather': {'prompt': 'Which city would you like the weather for?',
   'confirmation': 'Let me check the weather in {city}!'},
  'schedule_reminder': {'prompt': 'What would you like to be reminded about?',
   'confirmation': "Got it! I’ll remind you to '{task}' at {time}."}},
 'end_conversation': 'It was great helping you! 👋'}

In [5]:
import json
import os

CONFIG_PATH = "config"

def load_json(filename):
    path = os.path.join(CONFIG_PATH, filename)
    if not os.path.exists(path):
        print(f"❌ File not found: {path}")
        return {}
    with open(path, "r") as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            print(f"⚠️ Failed to parse JSON: {path}")
            return {}


def show_agent_personality():    
    profile = load_json("personality.json")
    if not profile:
        return
    print(f"🤖 Agent Name: {profile.get('name', 'N/A')}")
    print(f"💬 Style: {profile.get('style', 'N/A')}, Tone: {profile.get('tone', 'N/A')}")
    print("🌟 Core Values:", ", ".join(profile.get("core_values", [])))
    print("🔒 On Sensitive Topics:", profile.get("sensitive_response", 'N/A'))
    print("❓ On Unknown Requests:", profile.get("handle_unknown", 'N/A'))

def list_capabilities():
    caps = load_json("capabilities.json")
    if not caps:
        return
    print("\n🛠 Capabilities:")
    for cap in caps["capabilities"]:
        print(f"- {cap.get('task', 'Unknown')} → {cap.get('description', 'No description')}")
        print(f"  ✅ Success: {cap.get('success_criteria', 'N/A')}")
        print(f"  🔗 Uses API: {cap.get('required_api', 'None')}\n")

def show_conversation_flow(task=None):
    flow = load_json("ux_flow.json")
    if not flow:
        return
    if task:
        flow_data = flow.get("flows", {}).get(task)
        if flow_data:
            print(f"🧭 Flow for '{task}':")
            print("🗣 Prompt:", flow_data.get("prompt", "N/A"))
            print("✅ Confirmation:", flow_data.get("confirmation", "N/A"))
        else:
            print(f"⚠ No flow found for task '{task}'")
    else:
        print("🔄 General Start:", flow.get("start_conversation", "N/A"))
        print("😕 Fallback:", flow.get("fallback", "N/A"))
        print("👋 End:", flow.get("end_conversation", "N/A"))

# Run demo
if __name__ == "__main__":
    show_agent_personality()
    list_capabilities()
    show_conversation_flow()
    show_conversation_flow("get_weather")

❌ File not found: config\personality.json
❌ File not found: config\capabilities.json
❌ File not found: config\ux_flow.json
❌ File not found: config\ux_flow.json


In [8]:
INTENT_KEYWORDS = {
    "get_weather": ["weather", "forecast", "temperature"],
    "schedule_meeting": ["schedule", "meeting", "reminder", "set alarm"],
    "ask_question": ["who", "what", "where", "when", "why", "how"]
}

def classify_intent(user_input):
    user_input = user_input.lower()
    for intent, keywords in INTENT_KEYWORDS.items():
        if any(keyword in user_input for keyword in keywords):
            return intent
    return "unknown"

In [11]:
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        entities[ent.label_] = ent.text
    return entities

def get_location(doc):
    return [ent.text for ent in doc.ents if ent.label_ == "GPE"]

def get_time_date(doc):
    return [ent.text for ent in doc.ents if ent.label_ in ["DATE", "TIME"]]

def extract_details(text):
    doc = nlp(text)
    return {
         "locations": get_location(doc),
        "datetime": get_time_date(doc),
        "people": [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    }

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 2.4 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 2.4 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/12.8 MB 1.9 MB/s eta 0:00:06
     ------ --------------------------------- 2.1/12.8 MB 2.1 MB/s eta 0:00:06
     ------- -------------------------------- 2.4/12.8 MB 2.1 MB/s eta 0:00:06
     --------- ------------------------------ 3.1/12.8 MB 2.1 MB/s eta 0:00:05
     ----------- ---------------------------- 3.7/12.8 MB 2.2 MB/s eta 0:00:05
     ------------- -------------------------- 4.2/12.8 MB 2.3 MB/s eta 0:00:04
     -------------- ------------------------- 4.7/12.8 MB 2.3 MB/s eta 0:00:04
     --------------- ------------------------ 5.0/12.8 MB 2.2 MB/s eta 0:00:04
     ----------------- ---------------------- 5.5/12.8 MB 2.2 MB/s

In [12]:
class ContextManager:
    def __init__(self):
        self.sessions = {}

    def init(self, user_id):
        if user_id not in self.sessions:
            self.sessions[user_id] = {
                "intent": None,
                "entities": {},
                "history": []
            }

    def get(self, user_id):
        self.init(user_id)
        return self.sessions[user_id]

    def reset(self, user_id):
        self.sessions[user_id] = {
            "intent": None,
            "entities": {},
            "history": []
        }

    def update(self, user_id, intent=None, entities=None):
        self.init(user_id)
        if intent:
            self.sessions[user_id]["intent"] = intent
        if entities:
            self.sessions[user_id]["entities"].update(entities)
        self.sessions[user_id]["history"].append({
            "intent": intent,
            "entities": entities
        })


In [13]:
import os

# Create 'nlu' directory if it doesn't exist
os.makedirs("nlu", exist_ok=True)

# __init__.py
with open("nlu/__init__.py", "w") as f:
    f.write("# Init file for nlu module\n")

# intent_classifier.py
with open("nlu/intent_classifier.py", "w") as f:
    f.write('''
def classify_intent(text):
    text = text.lower()
    if "hello" in text or "hi" in text:
        return "greet"
    elif "book" in text and "flight" in text:
        return "book_flight"
    elif "weather" in text:
        return "check_weather"
    elif "bye" in text:
        return "goodbye"
    else:
        return "unknown"
''')

# entity_extractor.py
with open("nlu/entity_extractor.py", "w") as f:
    f.write('''
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_details(text):
    doc = nlp(text)
    return {
        "locations": [ent.text for ent in doc.ents if ent.label_ == "GPE"],
        "datetime": [ent.text for ent in doc.ents if ent.label_ in ["DATE", "TIME"]],
        "people": [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    }
''')

# context_manager.py
with open("nlu/context_manager.py", "w") as f:
    f.write('''
class ContextManager:
    def __init__(self):
        self.sessions = {}
        
    def init(self, user_id):
        if user_id not in self.sessions:
            self.sessions[user_id] = {
                "intent": None,
                "entities": {},
                "history": []
            }

    def get(self, user_id):
        self.init(user_id)
        return self.sessions[user_id]

    def reset(self, user_id):
        self.sessions[user_id] = {
            "intent": None,
            "entities": {},
            "history": []
        }
        
    def update(self, user_id, intent=None, entities=None):
        self.init(user_id)
        if intent:
            self.sessions[user_id]["intent"] = intent
        if entities:
            self.sessions[user_id]["entities"].update(entities)
        self.sessions[user_id]["history"].append({
            "intent": intent,
            "entities": entities
        })
''')

print("✅ nlu module files created successfully!")

✅ nlu module files created successfully!


In [ ]:
from nlu.intent_classifier import classify_intent
from nlu.entity_extractor import extract_details
from nlu.context_manager import ContextManager

context = ContextManager()
user_id = "user123"

print("🧠 Welcome to the NLU Test Agent")
print("Type 'exit' to quit\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    intent = classify_intent(user_input)
    entities = extract_details(user_input)

    context.update(user_id, intent, entities)
    
    print(f"🔎 Detected Intent: {intent}")
    print(f"📍 Entities: {entities}")
    print(f"🗂  Context: {context.get(user_id)}\n")


🧠 Welcome to the NLU Test Agent
Type 'exit' to quit



You:  set a reminder to call Dr.smith tomorrow at 4 PM in japan


🔎 Detected Intent: unknown
📍 Entities: {'locations': ['japan'], 'datetime': ['tomorrow', '4 PM'], 'people': []}
🗂  Context: {'intent': 'unknown', 'entities': {'locations': ['japan'], 'datetime': ['tomorrow', '4 PM'], 'people': []}, 'history': [{'intent': 'unknown', 'entities': {'locations': ['japan'], 'datetime': ['tomorrow', '4 PM'], 'people': []}}]}

